In [41]:
#import packages
import numpy as np
import pandas as pd
import os, sys
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn import svm
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [11]:
#EDA fucntion from previous work
def expl_data_analisys(source_frame):

    display(source_frame)
    source_frame.info()
    print()


    print ('\033[1m' + 'Null data amount' + '\033[0m')
    print()
    print(source_frame.isna().mean().sort_values(ascending = False).round(4) * 100)
    print()
    print('\033[1m' + 'Null data for float, int, %' + '\033[0m')
    print()
    for col in source_frame:
        if source_frame[col].dtypes in (float, int):
            print('{: >0}'.format(col), source_frame.loc[source_frame[col]==0, col].count())
    print()
    

    print('\033[1m' + 'Categorial variables' + '\033[1m')
    print()
    for col in source_frame:
        if source_frame[col].dtypes == object:
            print(('{: ^50}'.format(col)))
            display(source_frame[col].value_counts())
            print()


    print('\033[1m' + 'Quantitative variables' + '\033[1m')
    display(source_frame.describe())


In [55]:
#import files 
df=pd.read_csv('survey-lung-cancer.csv')
expl_data_analisys(df)

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,M,69,1,2,2,1,1,2,1,2,2,2,2,2,2,YES
1,M,74,2,1,1,1,2,2,2,1,1,1,2,2,2,YES
2,F,59,1,1,1,2,1,2,1,2,1,2,2,1,2,NO
3,M,63,2,2,2,1,1,1,1,1,2,1,1,2,2,NO
4,F,63,1,2,1,1,1,1,1,2,1,2,2,1,1,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,F,56,1,1,1,2,2,2,1,1,2,2,2,2,1,YES
305,M,70,2,1,1,1,1,2,2,2,2,2,2,1,2,YES
306,M,58,2,1,1,1,1,1,2,2,2,2,1,1,2,YES
307,M,67,2,1,2,1,1,2,2,1,2,2,2,1,2,YES


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   GENDER                 309 non-null    object
 1   AGE                    309 non-null    int64 
 2   SMOKING                309 non-null    int64 
 3   YELLOW_FINGERS         309 non-null    int64 
 4   ANXIETY                309 non-null    int64 
 5   PEER_PRESSURE          309 non-null    int64 
 6   CHRONIC DISEASE        309 non-null    int64 
 7   FATIGUE                309 non-null    int64 
 8   ALLERGY                309 non-null    int64 
 9   WHEEZING               309 non-null    int64 
 10  ALCOHOL CONSUMING      309 non-null    int64 
 11  COUGHING               309 non-null    int64 
 12  SHORTNESS OF BREATH    309 non-null    int64 
 13  SWALLOWING DIFFICULTY  309 non-null    int64 
 14  CHEST PAIN             309 non-null    int64 
 15  LUNG_CANCER            

M    162
F    147
Name: GENDER, dtype: int64


                   LUNG_CANCER                    


YES    270
NO      39
Name: LUNG_CANCER, dtype: int64


Quantitative variables


,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN
count,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000
mean,62.673139,1.563107,1.569579,1.498382,1.501618,1.504854,1.673139,1.556634,1.556634,1.556634,1.579288,1.640777,1.469256,1.556634
std,8.210301,0.496806,0.495938,0.500808,0.500808,0.500787,0.469827,0.497588,0.497588,0.497588,0.494474,0.480551,0.499863,0.497588
min,21.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,57.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,62.000000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000
75%,69.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,87.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


In [19]:
df.isnull().sum()

GENDER                   0
AGE                      0
SMOKING                  0
YELLOW_FINGERS           0
ANXIETY                  0
PEER_PRESSURE            0
CHRONIC DISEASE          0
FATIGUE                  0
ALLERGY                  0
WHEEZING                 0
ALCOHOL CONSUMING        0
COUGHING                 0
SHORTNESS OF BREATH      0
SWALLOWING DIFFICULTY    0
CHEST PAIN               0
LUNG_CANCER              0
dtype: int64

In [54]:
#null value removing with true and false

#findout empty value and replace it with anything else

df['GENDER'] = df['GENDER'].replace('M',1)
df['GENDER'] = df['GENDER'].replace('F',0)
df['LUNG_CANCER'] = df['LUNG_CANCER'].replace('YES',1)
df['LUNG_CANCER'] = df['LUNG_CANCER'].replace('NO',0)
df

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,1,69.0,1,2,2,1,1,2,1,2,2,2,2,2,2,1
1,1,74.0,2,1,1,1,2,2,2,1,1,1,2,2,2,1
2,0,59.0,1,1,1,2,1,2,1,2,1,2,2,1,2,0
3,1,63.0,2,2,2,1,1,1,1,1,2,1,1,2,2,0
4,0,63.0,1,2,1,1,1,1,1,2,1,2,2,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,0,56.0,1,1,1,2,2,2,1,1,2,2,2,2,1,1
305,1,70.0,2,1,1,1,1,2,2,2,2,2,2,1,2,1
306,1,58.0,2,1,1,1,1,1,2,2,2,2,1,1,2,1
307,1,67.0,2,1,2,1,1,2,2,1,2,2,2,1,2,1


In [59]:
def feature_selection_custome():
  filename = 'survey-lung-cancer.csv'
  names = ['GENDER','AGE','SMOKING','YELLOW_FINGERS','ANXIETY','PEER_PRESSURE','CHRONIC DISEASE','FATIGUE ','ALLERGY ','WHEEZING','ALCOHOL CONSUMING','COUGHING','SHORTNESS OF BREATH','SWALLOWING DIFFICULTY','CHEST PAIN']
  dataframe = pd.read_csv(filename, names=names)
  # dataframe["AGE"] = pd.to_numeric(dataframe["AGE"], downcast="float")
  array = dataframe.values
  X = array[:,0:14]
  Y = array[:,14]
  # feature extraction
  test = SelectKBest(score_func=f_classif, k=4)
  fit = test.fit(X, Y)
  # summarize scores
  set_printoptions(precision=3)
  print(fit.scores_)
  features = fit.transform(X)
  # summarize selected features
  print(features[0:5,:])

# feature_selection_custome()

In [33]:
#devided into dependent and independent variables
features = df [['GENDER','AGE','SMOKING','YELLOW_FINGERS','ANXIETY','PEER_PRESSURE','CHRONIC DISEASE','FATIGUE ','ALLERGY ','WHEEZING','ALCOHOL CONSUMING','COUGHING','SHORTNESS OF BREATH','SWALLOWING DIFFICULTY','CHEST PAIN']]
outcome = df ['LUNG_CANCER']

In [17]:
#check output
# features
# outcome

In [34]:
#scaling datasets into x and y


scaler=MinMaxScaler((-1,1))
x=scaler.fit_transform(features)
y=outcome

In [35]:

#train and test data
without_out_train,without_out_test,out_train,out_test = train_test_split(x, y, test_size=0.3, random_state=7)

In [36]:
#implementing logistic regression

logistic = LogisticRegression(random_state = 0)
logistic.fit(without_out_train,out_train)

logistic_pred = logistic.predict(without_out_test)
logistic_score = accuracy_score(out_test,logistic_pred)
precission_logistic = precision_score(out_test,logistic_pred)
recall_logistic = recall_score(out_test,logistic_pred)
f1_logistic = f1_score(out_test,logistic_pred)
print("Accuracy : ",logistic_score)
print("Confusion : ",confusion_matrix(out_test, logistic_pred))
print("Precission : ",precission_logistic)
print("Recall : ",recall_logistic)
print("F1 : ",f1_logistic)

Accuracy :  0.9139784946236559
Confusion :  [[10  6]
 [ 2 75]]
Precission :  0.9259259259259259
Recall :  0.974025974025974
F1 :  0.949367088607595


In [37]:
tree = DecisionTreeClassifier()
tree.fit(without_out_train,out_train)
tree_pred = tree.predict(without_out_test)
tree_score = accuracy_score(out_test,tree_pred)
precission_tree = precision_score(out_test,tree_pred)
recall_tree = recall_score(out_test,tree_pred)
f1_tree = f1_score(out_test,tree_pred)
print("Accuracy : ",tree_score)
print("Confusion : ",confusion_matrix(out_test, tree_pred))
print("Precission : ",precission_tree)
print("Recall : ",recall_tree)
print("F1 : ",f1_tree)

Accuracy :  0.8817204301075269
Confusion :  [[10  6]
 [ 5 72]]
Precission :  0.9230769230769231
Recall :  0.935064935064935
F1 :  0.9290322580645162


In [38]:
linear = LinearRegression()
linear.fit(without_out_train,out_train)
linear_pred = linear.predict(without_out_test)
linear_score = linear.score(without_out_test,out_test)
#precission_linear = precision_score(out_test,linear_pred)
#recall_linear = recall_score(out_test,linear_pred)
#f1_linear = f1_score(out_test,linear_pred)
print("Accuracy : ",linear_score)
# print("Confusion : ",confusion_matrix(out_test, linear_pred))
#print("Precission : ",precission_linear)
#print("Recall : ",recall_linear)
#print("F1 : ",f1_linear)

Accuracy :  0.4100342556792285


In [39]:
kn = KNeighborsClassifier()
kn.fit(without_out_train,out_train)
kn_pred = kn.predict(without_out_test)
kn_score = accuracy_score(out_test,kn_pred)
precission_kn = precision_score(out_test,kn_pred)
recall_kn = recall_score(out_test,kn_pred)
f1_kn = f1_score(out_test,kn_pred)
print("Accuracy : ",kn_score)
print("Confusion : ",confusion_matrix(out_test, kn_pred))
print("Precission : ",precission_kn)
print("Recall : ",recall_kn)
print("F1 : ",f1_kn)

Accuracy :  0.8709677419354839
Confusion :  [[ 7  9]
 [ 3 74]]
Precission :  0.891566265060241
Recall :  0.961038961038961
F1 :  0.9249999999999999


In [40]:
gbm = GradientBoostingClassifier()
gbm.fit(without_out_train,out_train)
gbm_pred = gbm.predict(without_out_test)
gbm_score = accuracy_score(out_test,gbm_pred)
precission_gbm = precision_score(out_test,gbm_pred)
recall_gbm = recall_score(out_test,gbm_pred)
f1_gbm = f1_score(out_test,gbm_pred)
print("Accuracy : ",gbm_score)
print("Confusion : ",confusion_matrix(out_test, gbm_pred))
print("Precission : ",precission_gbm)
print("Recall : ",recall_gbm)
print("F1 : ",f1_gbm)

Accuracy :  0.8924731182795699
Confusion :  [[ 9  7]
 [ 3 74]]
Precission :  0.9135802469135802
Recall :  0.961038961038961
F1 :  0.9367088607594937
